<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Preprocessing</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">


In [2]:
import config, src

import numpy as np
import pandas as pd

In [3]:
data = src.load_data(config.RAW_DATA_FILE)
data.info()

Données chargées avec succès.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50005 entries, 0 to 50004
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  float64
 1   artist_name       50000 non-null  object 
 2   track_name        50000 non-null  object 
 3   popularity        50000 non-null  float64
 4   acousticness      50000 non-null  float64
 5   danceability      50000 non-null  float64
 6   duration_ms       50000 non-null  float64
 7   energy            50000 non-null  float64
 8   instrumentalness  50000 non-null  float64
 9   key               50000 non-null  object 
 10  liveness          50000 non-null  float64
 11  loudness          50000 non-null  float64
 12  mode              50000 non-null  object 
 13  speechiness       50000 non-null  float64
 14  tempo             50000 non-null  object 
 15  obtained_date     50000 non-null  object 
 16  valence   

In [4]:
data.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [ ]:
data[data.isna().any(axis=1)]

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data = src.handle_missing_values(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 0 to 50004
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  float64
 1   artist_name       50000 non-null  object 
 2   track_name        50000 non-null  object 
 3   popularity        50000 non-null  float64
 4   acousticness      50000 non-null  float64
 5   danceability      50000 non-null  float64
 6   duration_ms       50000 non-null  float64
 7   energy            50000 non-null  float64
 8   instrumentalness  50000 non-null  float64
 9   key               50000 non-null  object 
 10  liveness          50000 non-null  float64
 11  loudness          50000 non-null  float64
 12  mode              50000 non-null  object 
 13  speechiness       50000 non-null  float64
 14  tempo             50000 non-null  object 
 15  obtained_date     50000 non-null  object 
 16  valence           50000 non-null  float64
 17

In [5]:
data['tempo'] = data['tempo'].replace('?', np.nan).astype(float)

for genre in data['music_genre'].unique():
    mean_tempo = data.loc[~ data['tempo'].isna() * data['music_genre'] == genre, 'tempo'].mean()
    data.loc[data['tempo'].isna() * data['music_genre'] == genre, 'tempo'] = mean_tempo

In [7]:
for genre in config.CLASSES.values():
    mean_duration = data['duration_ms'].loc[(data['duration_ms'] != -1) & (data['music_genre'] == genre)].mean()
    data.loc[(data['music_genre'] == genre) & (data['duration_ms'] == -1), 'duration_ms'] = mean_duration

In [8]:
data_combined_rap_hiphop = data.copy()
data_combined_rap_hiphop['music_genre'] = data_combined_rap_hiphop['music_genre'].replace({'Hip-Hop' : 'Rap/Hip-Hop', 'Rap': 'Rap/Hip-Hop'})
print(data_combined_rap_hiphop['music_genre'].unique())

['Electronic' 'Anime' 'Jazz' 'Alternative' 'Country' 'Rap/Hip-Hop' 'Blues'
 'Rock' 'Classical']


In [9]:
data = src.encode_categorical(data, columns=['key', 'mode', 'music_genre'])
data.drop(columns=['instance_id', 'artist_name', 'track_name', 'obtained_date'], inplace=True)
data.head()

Encodage de la variable 'key' : A -> 0 | A# -> 1 | B -> 2 | C -> 3 | C# -> 4 | D -> 5 | D# -> 6 | E -> 7 | F -> 8 | F# -> 9 | G -> 10 | G# -> 11
Encodage de la variable 'mode' : Major -> 0 | Minor -> 1
Encodage de la variable 'music_genre' : Alternative -> 0 | Anime -> 1 | Blues -> 2 | Classical -> 3 | Country -> 4 | Electronic -> 5 | Hip-Hop -> 6 | Jazz -> 7 | Rap -> 8 | Rock -> 9


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,27.0,0.00468,0.652,270703.43126,0.941,0.79200,1,0.115,-5.201,1,0.0748,100.889,0.759,5
1,31.0,0.01270,0.622,218293.00000,0.890,0.95000,5,0.124,-7.043,1,0.0300,115.002,0.531,5
2,28.0,0.00306,0.620,215613.00000,0.755,0.01180,11,0.534,-4.617,0,0.0345,127.994,0.333,5
3,34.0,0.02540,0.774,166875.00000,0.700,0.00253,4,0.157,-4.498,0,0.2390,128.014,0.270,5
4,32.0,0.00465,0.638,222369.00000,0.587,0.90900,9,0.157,-6.266,0,0.0413,145.036,0.323,5


In [10]:
src.save_data(data, 'processed_data')

Données sauvegardées avec succès.


In [11]:
is_rap_hiphop = data_combined_rap_hiphop['music_genre'] == 'Rap/Hip-Hop'
n_dups_rap_hiphop = data_combined_rap_hiphop[is_rap_hiphop].duplicated(subset=['artist_name', 'track_name']).sum()
print(f'Il y a {n_dups_rap_hiphop} titres qui sont présents plusieurs fois pour le genre "Rap/Hip-Hop".')

Il y a 1588 titres qui sont présents plusieurs fois pour le genre "Rap/Hip-Hop".


In [12]:
data_rap_hiphop = data_combined_rap_hiphop[is_rap_hiphop].drop_duplicates(subset=['artist_name', 'track_name'])
data_other_genres = data_combined_rap_hiphop[~is_rap_hiphop]
data_combined_rap_hiphop = pd.concat([data_rap_hiphop, data_other_genres], ignore_index=True)

data_combined_rap_hiphop.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,61813.0,Jeezy,Hustlaz Ambition,51.0,0.000469,0.683,219933.00000,0.769,0.002260,B,0.303,-7.108,Minor,0.2270,142.827,4-Apr,0.475,Rap/Hip-Hop
1,28170.0,Smino,L.M.F.,63.0,0.354000,0.898,218927.02139,0.677,0.000005,C#,0.449,-6.907,Major,0.3620,155.020,4-Apr,0.864,Rap/Hip-Hop
2,69366.0,Kid Ink,F With U (feat. Ty Dolla $ign),61.0,0.400000,0.578,179533.00000,0.528,0.000000,G#,0.149,-7.169,Major,0.2740,98.555,4-Apr,0.622,Rap/Hip-Hop
3,46274.0,Breaking Benjamin,I Will Not Bow,71.0,0.000192,0.543,218927.02139,0.952,0.000008,G#,0.380,-3.650,Major,0.0668,107.942,4-Apr,0.591,Rap/Hip-Hop
4,83672.0,The Game,Sue You,17.0,0.003550,0.589,181441.00000,0.721,0.032500,G,0.193,-12.145,Major,0.1930,88.049,4-Apr,0.480,Rap/Hip-Hop


In [13]:
data_combined_rap_hiphop = src.encode_categorical(data_combined_rap_hiphop, columns=['key', 'mode', 'music_genre'])
data_combined_rap_hiphop.drop(columns=['instance_id', 'artist_name', 'track_name', 'obtained_date'], inplace=True)
data_combined_rap_hiphop.head()

Encodage de la variable 'key' : A -> 0 | A# -> 1 | B -> 2 | C -> 3 | C# -> 4 | D -> 5 | D# -> 6 | E -> 7 | F -> 8 | F# -> 9 | G -> 10 | G# -> 11
Encodage de la variable 'mode' : Major -> 0 | Minor -> 1
Encodage de la variable 'music_genre' : Alternative -> 0 | Anime -> 1 | Blues -> 2 | Classical -> 3 | Country -> 4 | Electronic -> 5 | Jazz -> 6 | Rap/Hip-Hop -> 7 | Rock -> 8


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,51.0,0.000469,0.683,219933.00000,0.769,0.002260,2,0.303,-7.108,1,0.2270,142.827,0.475,7
1,63.0,0.354000,0.898,218927.02139,0.677,0.000005,4,0.449,-6.907,0,0.3620,155.020,0.864,7
2,61.0,0.400000,0.578,179533.00000,0.528,0.000000,11,0.149,-7.169,0,0.2740,98.555,0.622,7
3,71.0,0.000192,0.543,218927.02139,0.952,0.000008,11,0.380,-3.650,0,0.0668,107.942,0.591,7
4,17.0,0.003550,0.589,181441.00000,0.721,0.032500,10,0.193,-12.145,0,0.1930,88.049,0.480,7


In [14]:
src.save_data(data_combined_rap_hiphop, 'processed_data_combined_rap_hiphop')

Données sauvegardées avec succès.


---